# 导入一些必要的库

In [1]:
#-*- coding:utf-8 -*-
import sys
import numpy as np
import pandas as pd
from random import random, randint

# 所用的几个库的版本

In [2]:
print (sys.version)
print (np.__version__)
print (pd.__version__)

3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
1.12.1
0.20.1


# 将数据从已经整理好的csv文件中读入

In [3]:
df = pd.read_csv('collectedData.csv', index_col=[0])
df.head()

,theta_upper,h_upper,theta_lower,h_lower,average
1,90.0,1.0,66.0,1.5,2.83
2,90.0,1.0,66.0,1.6,2.73
3,90.0,1.0,66.0,1.7,2.68
4,90.0,1.0,66.0,1.8,2.68
5,90.0,1.0,66.0,1.9,2.62


# 读入后的数据类型为pandas.DataFrame

In [4]:
type(df)

pandas.core.frame.DataFrame

# 对数据做进一步的处理，以符合算法的需要
## 这一步非常重要，刚开始的模型得到的结果一直不理想，我都想要换模型了，但对数据进行了Feature scaling 之后，发现效果出奇的好~

In [5]:
# 将数据转换成 numpy.array 类型，方便后续的运算
data = np.array(df)

# 加入数值全为 1 的一列，作为常数项，及 x0 （可参见斯坦福ML第4章），这个也很重要，没有这部误差挺大的，模型不能用
dataOne = np.ones((data.shape[0], 1))

# feature scaling （参见斯坦福ML第4章），极为重要的一步！
a = data[:, :-1].max(axis = 0)
b = data[:, :-1].min(axis = 0)
c = np.array([round(x, 1) for x in data[:, :-1].mean(axis=0)])
c = np.append(c, [0])
dataAfterFeatureScaling = (data - np.ones((data.shape[0], data.shape[1])) * c) / np.append((a - b), [1])
dataAfterFeatureScaling = np.hstack((dataOne, dataAfterFeatureScaling))

# 最后整理得到的数据为 numpy.array 类型，下面的是前5行值的展示

In [6]:
dataAfterFeatureScaling[:5,:]

array([[ 1.        ,  0.20392157,  0.        ,  0.00392157, -0.55555556,
         2.83      ],
       [ 1.        ,  0.20392157,  0.        ,  0.00392157, -0.44444444,
         2.73      ],
       [ 1.        ,  0.20392157,  0.        ,  0.00392157, -0.33333333,
         2.68      ],
       [ 1.        ,  0.20392157,  0.        ,  0.00392157, -0.22222222,
         2.68      ],
       [ 1.        ,  0.20392157,  0.        ,  0.00392157, -0.11111111,
         2.62      ]])

# 最为初始的线性模型公式计算版本，此处效果极差，因为数据是非线性的，所以后面采用了局部加权线性模型，可解决非线性问题

In [7]:
def standRegres(xArr,yArr):
    """
    用公式计算的标准线性回归模型
    """
    xMat = np.mat(xArr); yMat = np.mat(yArr)
    xTx = xMat.T*xMat
    if np.linalg.det(xTx) == 0.0:
        print ("This matrix is singular, cannot do inverse")
        return
    ws = xTx.I * (xMat.T*yMat)
    return ws

# 这次采用的回归模型——局部加权线性回归模型
## 效果很好~
## k为此模型中需要调整的参数
## 仍有可以改进的地方，但在此处数据规模不大，已经可以得到理想结果了，若要针对大数据可以改进

In [8]:
def lwlr(testPoint,xArr,yArr,k= 0.12):
    """
    局部加权线性回归模型（参见Machine learning in action中的第8章）
    """
    xMat = np.mat(xArr); yMat = np.mat(yArr)
    m = np.shape(xMat)[0]
    weights = np.mat(np.eye((m)))
    for j in range(m):
        diffMat = testPoint - xMat[j,:]
        weights[j,j] = np.exp(diffMat*diffMat.T/(-2.0*k**2))
    xTx = xMat.T * (weights * xMat)
    if np.linalg.det(xTx) == 0.0:
        print ("This matrix is singular, cannot do inverse")
        print (testPoint)
        return
    ws = xTx.I * (xMat.T * (weights * yMat))
    return testPoint * ws

# 将数据划分为95%的训练集和5%的测试集
## 这样划分是因为 h_upper和h_lower的特征数据集较少，测试集占的太多会导致训练集中无法得到h_upper和h_lower的特征

In [9]:
def divideData(data, p = 0.05):
    """
    划分数据为训练集和测试集（参见Programming collective intelligence的第8章）
    """
    trainSet = []
    testSet = []
    for row in data:
        if random() < p:
            testSet.append(row)
        else:
            trainSet.append(row)
    return trainSet, testSet

In [10]:
def testAlgorithm(alg, trainSet, testSet, k):
    """
    利用欧式距离的平方计算算法的平均variance error
    """
    error = 0.0
    trainSet = np.array(trainSet)
    testSet = np.array(testSet)
    count = 0
    for row in testSet:
        guess = alg(row[:-1], trainSet[:,:-1], trainSet[:, -1:], k)
        if guess is not None:
            count += 1
            error += (row[-1] - float(guess))**2
    if count != 0:
        return error / count, count
    else:
        return

# 利用交叉验证计算来算生成trials组随机trainSet和testSet后模型的平均误差

In [11]:
def crossValidate(alg, data, k = 0.12, trials = 100, p = 0.05):
    """
    利用交叉验证计算模型的平均误差
    """
    error = 0.0
    count = 0
    for i in range(trials):
        tempError = 0.0
        trainSet, testSet = divideData(data, p)
        res = testAlgorithm(alg, trainSet, testSet, k)
        if res is not None:
            tempError, c = res
            count += 1
            error += tempError
    if count != 0:
        return error / count, count
    else:
        return

# 在0.01-1.00中以0.01的步长，寻找能使得误差最小的k值作为最有的k
## 反映到数据上，回归模型的干涉量大概有0.03的误差，也就是说如果干涉量为2.56%，则预测的干涉量为2.56±0.03%的样子
## 下面计算需要大概2-3分钟，出现This matrix is singular, cannot do inverse的情况是因为k的取值过小了，大于0.05之后一般不会出现这种情况，这个不用理会就好

In [12]:
x = np.arange(0.01, 1, 0.01)

minVal = 100
bestK = 0.01
for guessK in x:
    t = crossValidate(lwlr, dataAfterFeatureScaling, guessK)
    if t[0] < minVal:
        minVal = t[0]
        bestK = guessK

# minVal为平均误差，bestK为需要寻找的最优值k
print (minVal**(0.5), bestK)

This matrix is singular, cannot do inverse
[ 1.          0.20392157  0.          0.00392157 -0.33333333]
This matrix is singular, cannot do inverse
[ 1.          0.20392157  0.42857143  0.00392157  0.        ]
This matrix is singular, cannot do inverse
[ 1.          0.20392157  0.         -0.36862745  0.        ]
This matrix is singular, cannot do inverse
[ 1.          0.20392157  0.          0.37647059  0.        ]
This matrix is singular, cannot do inverse
[ 1.         -0.7372549   0.          0.00392157  0.        ]
This matrix is singular, cannot do inverse
[ 1.         -0.58039216  0.          0.00392157  0.        ]
This matrix is singular, cannot do inverse
[ 1.         -0.34509804  0.          0.00392157  0.        ]
This matrix is singular, cannot do inverse
[ 1.          0.20392157  0.         -0.25098039  0.        ]
This matrix is singular, cannot do inverse
[ 1.         -0.71764706  0.          0.00392157  0.        ]
This matrix is singular, cannot do inverse
[ 1.        

# 通过上述方法已经建立了一个比较理想的回归模型，说白了就是调整了一个参数k，这样以来，现在随便放一组点进去就可以得到预期一个比较理想的预测值了
## 接下来要做的就是寻找最优解，也就是干涉量最大的那组解
## 此处利用了遗传算法来求解最优值

In [13]:
a

array([ 91. ,   1.5,  91. ,   2.4])

In [14]:
b

array([ 40. ,   0.8,  40. ,   1.5])

In [15]:
c[:-1]

array([ 79.6,   1. ,  65.8,   2. ])

In [16]:
def costf(point, trainSet, bestK):
    """
    代价函数
    """
    point = np.array(point)
    # 此处的a, b, c为之前计算的值
    meanVal = c[:-1]
    maxVal = a
    minVal = b
    xArr = (point - meanVal) / (maxVal - minVal)
    xArr = np.append([1.0], xArr)
    predictVal = lwlr(xArr, trainSet[:, :-1], trainSet[:, -1:], bestK)
    if predictVal is not None:
        return 100.0 / (float(predictVal) + 0.001), float(predictVal)
    else:
        return

In [17]:
def geneticoptimize(domain,costf,step,trainSet,bestK,popsize=50,mutprob=0.2,elite=0.2,maxiter=50):
    """
    遗传算法
    """
    # Mutation Operation
    def mutate(vec):
        i = randint(0, len(domain) - 1)
        if random( ) < 0.5 and vec[i] > domain[i][0]:
            return vec[0:i] + [vec[i] - step[i]] + vec[i+1:]
        elif vec[i] < domain[i][1]:
            return vec[0:i] + [vec[i] + step[i]] + vec[i+1:]

    # Crossover Operation
    def crossover(r1,r2):
        i = randint(1, len(domain) - 2)
        return r1[0:i]+r2[i:]
    
    # Build the initial population
    pop=[]
    for i in range(popsize):
        vec = [round(randint(domain[i][0] / step[i], domain[i][1] / step[i]) * step[i], 1) for i in range(len(domain))]
        pop.append(vec)
    
    # How many winners from each generation?
    topelite = int(elite * popsize)
    
    # Main loop
    for i in range(maxiter):
        scores = []
        for v in pop:
            costVal = costf(v, trainSet, bestK)
            if costVal is not None:
                costVal = costVal[0]
                scores.append((costVal, v))
        scores.sort( )
        ranked = [v for (s,v) in scores]
        
        # Start with the pure winners
        pop = ranked[0: topelite]
        
        # Add mutated and bred forms of the winners
        while len(pop) < popsize:
            if random( ) < mutprob:
                # Mutation
                c = randint(0, topelite)
                pop.append(mutate(ranked[c]))
            else:
                # Crossover
                c1 = randint(0, topelite)
                c2 = randint(0, topelite)
                pop.append(crossover(ranked[c1], ranked[c2]))
        
        # Print current best score
        print (scores[0][0])
    return scores[0][1]

In [18]:
# 特征值的取值范围
domain = [(40,90), (0.8,1.5), (40,90), (1.5, 2.5)]

#步长
step = [1.0, 0.1, 1.0, 0.1]

In [19]:
bestK

0.11

# 利用遗传算法，借助线性加权回归模型计算最佳的点

In [21]:
optimalPoint = geneticoptimize(domain, costf, step, dataAfterFeatureScaling, bestK, maxiter=30)
optimalPoint

26.764897627799463
25.464308578382685
25.464308578382685
25.351482255622802
25.314474410927325
25.203043246385338
25.16140331986603
25.066592765968462
24.94862116377058
24.878631205770585
24.792073548756857
24.792073548756857
24.77814554290595
24.77814554290595
24.77814554290595
24.77814554290595
24.77814554290595
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077
24.745370843127077


[73.0, 0.8, 44.0, 1.5]

# 上面的遗传算法输出的一大串为每次迭代时代价函数的代价值，数值大小没有意义，只是越小越好
## 最后来看下最优点的干涉量是多少

In [22]:
# 看看最佳点计算得到的最佳值是多少
costf(optimalPoint, dataAfterFeatureScaling, bestK)[1]

4.040159885376079